In [1]:
import os
from time import time
from pathlib import Path
import pandas as pd 
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import Dataset
import accelerate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

Obtenemos la carpeta actual:

In [2]:
current_dir = Path.cwd()
DATASETS_LOCATION = os.path.join(current_dir.parent.parent.parent, 'datasets')
MODELS_LOCATION = os.path.join(current_dir.parent.parent, 'models')

Cargamos el dataset en un dataframe:

In [3]:
df_disease_sp = pd.read_excel(os.path.join(DATASETS_LOCATION, 'disease_nlp_esp.xlsx'))
df_disease_sp.head(3)

,Síntomas,Enfermedad
0,Me siento cansado todo el tiempo y he notado q...,Anemia
1,Últimamente me falta el aire incluso después d...,Anemia
2,"Durante las últimas semanas, me siento extrema...",Anemia


In [4]:
print(df_disease_sp['Enfermedad'].value_counts())


Enfermedad
Anemia       100
Talasemia    100
Trombosis    100
Diabetes     100
Name: count, dtype: int64


Observamos que ya las clases están bien distribuidas. Dividimos el dataset en conjunto de entrenamiento y prueba, y después dividieremos el conujunto de entrenamiento en conjunto de entrenamiento y conjunto de validación:

In [5]:
train_df, test_df = train_test_split(df_disease_sp, test_size=0.2, random_state=42, stratify=df_disease_sp['Enfermedad'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Enfermedad'])

print("\nConjunto de entrenamiento:")
print(train_df['Enfermedad'].value_counts())

print("\nConjunto de validación:")
print(val_df['Enfermedad'].value_counts())

print("\nConjunto de prueba:")
print(test_df['Enfermedad'].value_counts())


Conjunto de entrenamiento:
Enfermedad
Trombosis    64
Anemia       64
Talasemia    64
Diabetes     64
Name: count, dtype: int64

Conjunto de validación:
Enfermedad
Trombosis    16
Talasemia    16
Anemia       16
Diabetes     16
Name: count, dtype: int64

Conjunto de prueba:
Enfermedad
Talasemia    20
Diabetes     20
Anemia       20
Trombosis    20
Name: count, dtype: int64


Codificamos las clases de 'Enfermedad', utilizando LabelEncoder:

In [6]:
label_encoder = LabelEncoder()

train_df['label'] = label_encoder.fit_transform(train_df['Enfermedad'])
val_df['label'] = label_encoder.transform(val_df['Enfermedad'])
test_df['label'] = label_encoder.transform(test_df['Enfermedad'])

train_df.head()

,Síntomas,Enfermedad,label
281,"He notado que mi pierna está hinchada, y las v...",Trombosis,3
22,"Estoy teniendo muchas infecciones, y mi piel s...",Anemia,0
146,"Últimamente me siento muy débil, y mi piel est...",Talasemia,2
70,He notado que el blanco de mis ojos tiene un t...,Anemia,0
376,"Tengo la vista borrosa, siempre tengo sed y pa...",Diabetes,1


## Entrenamiento

Definamos la función de métricas:

In [7]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average='macro'),
        "recall": recall_score(p.label_ids, preds, average='macro'),
        "f1": f1_score(p.label_ids, preds, average='macro'),
    }

Definamos el modelo y el tokenizer. Utilizaremos el modelo preentrenado de hugging face 'BERTIN':

In [8]:
model_checkpoint = "bertin-project/bertin-roberta-base-spanish"  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Preprocesamos los datos
def preprocess_function(examples):
    return tokenizer(examples['Síntomas'], examples['Enfermedad'], padding=True, truncation=True, return_tensors="pt")

Convertimos el dataframe en un dataset de hugging face, y lo tokenizamos:

In [10]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(val_df)

train_preprocessed_dataset = train_dataset.map(preprocess_function, batched=True)
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)
val_preprocessed_dataset = validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,              # Tasa de aprendizaje: se ha probado 5e-6, 1e-5, 5e-5 y 2e-5
    per_device_train_batch_size=32,   # Tamaño del batch para entrenamiento: se ha probado 8, 16, 32, 64
    per_device_eval_batch_size=32,    # Tamaño del batch para evaluación
    num_train_epochs=4,
    weight_decay=0.3,               # Decaimiento de peso: hemos probado 0.01, 0.1, 0.2 e 0.3 e 0.4 y este era el mejor
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_preprocessed_dataset,
    eval_dataset=val_preprocessed_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics   # Función para calcular las métricas
)

c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\maria\AppData\Local\Temp\ipykernel_17924\3271498588.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Training

Preprocesamos el conjunto de entrenamiento:

In [12]:
train_preprocessed_dataset

Dataset({
    features: ['Síntomas', 'Enfermedad', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 256
})

Y entrenamos nuestro modelo:

In [13]:
start = time()

trainer.train()

end = time()
print(f">>>>>>>>>>>>> elapsed time: {(end-start)/60:.0f}m")

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.1184666156768799, 'eval_accuracy': 0.875, 'eval_precision': 0.9082880434782609, 'eval_recall': 0.875, 'eval_f1': 0.8737463813068652, 'eval_runtime': 2.7965, 'eval_samples_per_second': 22.885, 'eval_steps_per_second': 0.715, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.6819532513618469, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 2.6387, 'eval_samples_per_second': 24.254, 'eval_steps_per_second': 0.758, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.40040576457977295, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 3.2715, 'eval_samples_per_second': 19.563, 'eval_steps_per_second': 0.611, 'epoch': 3.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.29349562525749207, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 4.2573, 'eval_samples_per_second': 15.033, 'eval_steps_per_second': 0.47, 'epoch': 4.0}
{'train_runtime': 182.018, 'train_samples_per_second': 5.626, 'train_steps_per_second': 0.176, 'train_loss': 0.7679097056388855, 'epoch': 4.0}
>>>>>>>>>>>>> elapsed time: 3m


## Evaluación

Evaluamos nuestro modelo en el conjunto de prueba:

In [14]:
# Preprocesamos el conjunto de prueba
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)

# Evaluamos el modelo en el conjunto de prueba
results = trainer.evaluate(test_preprocessed_dataset)

# Imprimimos las métricas de evaluación
print("Resultados de la evaluación en el conjunto de prueba:")
print(f"Exactitud: {results['eval_accuracy']:.4f}")
print(f"Precisión: {results['eval_precision']:.4f}")
print(f"Recall: {results['eval_recall']:.4f}")
print(f"F1-Score: {results['eval_f1']:.4f}")

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Resultados de la evaluación en el conjunto de prueba:
Exactitud: 1.0000
Precisión: 1.0000
Recall: 1.0000
F1-Score: 1.0000


Ahora, guardamos el modelo y el tokenizer:

In [15]:
save_directory = os.path.join(MODELS_LOCATION, 'disease_classification_spanish_nlp')
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Modelo y tokenizer guardados en el directorio {save_directory}")

Modelo y tokenizer guardados en el directorio c:\Users\maria\Desktop\universidad\master\TFM\tfm\src\models\disease_classification_spanish_nlp


Pasemos ahora a definir algunas frases de ejemplo para el test, que puedan confunir a nuestro modelo:

In [16]:
sample_sentences = [
    {"Síntomas": "Me siento cansado todo el tiempo, como si no tuviera energía para nada.", "Enfermedad": "Anemia"},
    {"Síntomas": "Mi hijo no crece como debería y siempre se queja de estar débil.", "Enfermedad": "Talasemia"},
    {"Síntomas": "Tengo dolores en los huesos y me siento agotado incluso después de descansar.", "Enfermedad": "Talasemia"},
    {"Síntomas": "Tengo mucha sed todo el tiempo y no dejo de ir al baño.", "Enfermedad": "Diabetes"},
    {"Síntomas": "Tengo un dolor punzante y la pierna está hinchada y roja.", "Enfermedad": "Trombosis"},
    {"Síntomas": "He bajado de peso sin razón y me siento muy cansado.", "Enfermedad": "Diabetes"}
]

# Convirtamos las frases de ejemplo a un dataframe
sample_df = pd.DataFrame(sample_sentences)

# Tokenicemos las frases de ejemplo
sample_dataset = Dataset.from_pandas(sample_df)
sample_preprocessed_dataset = sample_dataset.map(preprocess_function, batched=True)

# Efectuemos las predicciones
sample_predictions = trainer.predict(sample_preprocessed_dataset)
sample_preds = np.argmax(sample_predictions.predictions, axis=1)
sample_df["Enfermedad predicha"] = label_encoder.inverse_transform(sample_preds)

print("Resultados de las predicciones en las frases de ejemplo:")
sample_df

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Resultados de las predicciones en las frases de ejemplo:


,Síntomas,Enfermedad,Enfermedad predicha
0,"Me siento cansado todo el tiempo, como si no t...",Anemia,Anemia
1,Mi hijo no crece como debería y siempre se que...,Talasemia,Talasemia
2,Tengo dolores en los huesos y me siento agotad...,Talasemia,Talasemia
3,Tengo mucha sed todo el tiempo y no dejo de ir...,Diabetes,Diabetes
4,Tengo un dolor punzante y la pierna está hinch...,Trombosis,Trombosis
5,He bajado de peso sin razón y me siento muy ca...,Diabetes,Diabetes
